In [ ]:
#!pip install nltk

In [1]:
import pandas as pd
import numpy as np
import os
from nltk.tokenize import sent_tokenize
from transformers import BertTokenizerFast, BertForSequenceClassification
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /home/users/rldall/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
# specify GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load the BERT tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')

#import bert
path_model = ('saved_weights_3lab_8batch.pt')
loaded_model =  BertForSequenceClassification.from_pretrained('bert-base-cased',num_labels = 3)
loaded_model.load_state_dict(torch.load(path_model))

loaded_model.eval()
loaded_model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [5]:
# sent preprocessing
def get_sent_argfeat(sent,tokenizer,model):
    # We need Token IDs and Attention Mask for inference on the new sentence
    test_ids = []
    test_attention_mask = []

    # Apply the tokenizer
    encoding = tokenizer.encode_plus(
                        sent,
                        add_special_tokens = True,
                        max_length = 256,
                        padding = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

    # Extract IDs and Attention Mask
    test_ids.append(encoding['input_ids'])
    test_attention_mask.append(encoding['attention_mask'])
    test_ids = torch.cat(test_ids, dim = 0)
    test_attention_mask = torch.cat(test_attention_mask, dim = 0)
    
    with torch.no_grad():
        output = model(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))
    
    pred = np.argmax(output.logits.cpu().numpy()).flatten().item()
    
#    del test_ids, test_attention_mask, output, sent
    
    return pred

In [8]:
files = [
 'nyt-edu.txt',
 'nyt-med.txt',
 'nyt-fin.txt',
 'nyt-mil.txt',
 'nyt-pol.txt'
]

In [ ]:
##df_list = []

##for f in files:
##    df = pd.read_csv('output_txt/'+f,sep='\t',header=None)
##    df_list.append(df)

In [9]:
N = 100
result_list = []

for f in files:
##for df in df_list:
    df = pd.read_csv('output_txt/'+f,sep='\t',header=None)
    
    print('file name:',f)
##    print(df_list.index(df))
    
    count_long = 0
    pred_list = []
    error_list = []
    
    print('reset count_long and pred_list')

    for idx,row in df.iterrows():
        
        if (idx+1) % 200 == 0:
            print ('finished {} sentences from {}'.format(idx+1,len(df)))
        
        t = row[1]
        sent_token = sent_tokenize(t)
        
        if idx == 0:
            print ('tokenize the first sentence')

    
        # pad till N
        if len(sent_token) > N:
            try:
                #print(len(sent_token))
                pred_text = [get_sent_argfeat(sent,tokenizer,loaded_model)+1 for sent in sent_token[:100]]
                count_long += 1
            except:
                print('Error line:',idx)
                error_list.append(idx)
        else:
            try:
                #print(len(sent_token))
                pred_text = [get_sent_argfeat(sent,tokenizer,loaded_model)+1 for sent in sent_token]# + [0] * (N-len(sent_token))
            except:
                print('Error line:',idx)
                error_list.append(idx)
        
        if idx == 0:
            print('got the first sentence')
        
        pred_list.append(pred_text)
    
    #    print (pred_text)
    
    print('long articles:',count_long,'from',len(df))
    print('percent of long articles:', count_long/(count_long+len(df)))
    
    df[2] = pred_list
    df = df.drop(error_list)
    df.to_csv('output_txt/'+f[:-4]+'-argfeat-3.txt',sep='\t',header=None,index=False)
##    result_list.append(df)
    
    print('saved')

    torch.cuda.empty_cache()

file name: nyt-edu.txt
reset count_long and pred_list
tokenize the first sentence
got the first sentence


/home/users/rldall/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2354: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


finished 200 sentences from 1881
finished 400 sentences from 1881
Error line: 575
finished 600 sentences from 1881
finished 800 sentences from 1881
finished 1000 sentences from 1881
finished 1200 sentences from 1881
finished 1400 sentences from 1881
finished 1600 sentences from 1881
Error line: 1729
finished 1800 sentences from 1881
long articles: 48 from 1881
percent of long articles: 0.024883359253499222
saved
file name: nyt-med.txt
reset count_long and pred_list
tokenize the first sentence


/home/users/rldall/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2354: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


got the first sentence
finished 200 sentences from 1743
finished 400 sentences from 1743
finished 600 sentences from 1743
finished 800 sentences from 1743
finished 1000 sentences from 1743
finished 1200 sentences from 1743
finished 1400 sentences from 1743
finished 1600 sentences from 1743
long articles: 44 from 1743
percent of long articles: 0.024622271964185788
saved
file name: nyt-fin.txt
reset count_long and pred_list
tokenize the first sentence


...

got the first sentence
finished 200 sentences from 3100
finished 400 sentences from 3100
finished 600 sentences from 3100
finished 800 sentences from 3100
finished 1000 sentences from 3100
finished 1200 sentences from 3100
finished 1400 sentences from 3100
finished 1600 sentences from 3100
finished 1800 sentences from 3100
Error line: 1977
finished 2000 sentences from 3100
finished 2200 sentences from 3100
finished 2400 sentences from 3100
finished 2600 sentences from 3100
finished 2800 sentences from 3100
Error line: 2916
finished 3000 sentences from 3100
long articles: 39 from 3100
percent of long articles: 0.012424338961452692
saved
file name: nyt-mil.txt
reset count_long and pred_list
tokenize the first sentence


/home/users/rldall/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2354: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


got the first sentence
finished 200 sentences from 2132
finished 400 sentences from 2132
finished 600 sentences from 2132
finished 800 sentences from 2132
finished 1000 sentences from 2132
finished 1200 sentences from 2132
finished 1400 sentences from 2132
finished 1600 sentences from 2132
finished 1800 sentences from 2132
finished 2000 sentences from 2132
long articles: 19 from 2132
percent of long articles: 0.008833100883310088
saved
file name: nyt-pol.txt
reset count_long and pred_list
tokenize the first sentence


/home/users/rldall/.local/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2354: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


got the first sentence
finished 200 sentences from 6886
finished 400 sentences from 6886
finished 600 sentences from 6886
finished 800 sentences from 6886
finished 1000 sentences from 6886
finished 1200 sentences from 6886
finished 1400 sentences from 6886
finished 1600 sentences from 6886
finished 1800 sentences from 6886
finished 2000 sentences from 6886
finished 2200 sentences from 6886
finished 2400 sentences from 6886
finished 2600 sentences from 6886
finished 2800 sentences from 6886
finished 3000 sentences from 6886
finished 3200 sentences from 6886
finished 3400 sentences from 6886
finished 3600 sentences from 6886
finished 3800 sentences from 6886
finished 4000 sentences from 6886
finished 4200 sentences from 6886
finished 4400 sentences from 6886
finished 4600 sentences from 6886
finished 4800 sentences from 6886
finished 5000 sentences from 6886
finished 5200 sentences from 6886
finished 5400 sentences from 6886
finished 5600 sentences from 6886
finished 5800 sentences from 